# Twitter Analysis Pipeline

![](https://www.evernote.com/l/AAFtiWq4lUNNiqyswF72dUV3f_7Geq_TcykB/image.png)

In [ ]:
! pip install boto3 pymongo twitter --quiet

In [ ]:
import lib 

## Write Tweets to S3

### Create a Tweet Iterator

In [ ]:
token = '2379975860-fvNwQM9Kk9bL4A6AyTNdGSSJAKuE9dDnTEISg3q' # access token
token_secret = 'JPYjiwcOWnGy6nuJEvDt9owkKaQvXcvN2sryA8wl3kQxL' # access token secre
consumer_key = 'iJm4IBd4wmfJJjOTvQiMOrEXf'
consumer_secret = '9a4G3kwY1w3OmekWdmQGMo2nxLuxTxttjDQrfsRuvI9rUv29O6'   # API secret
bounding_box    = "-118.5137323688,34.0001996344,-118.4702449172,34.0331651696"

tweet_iterator  = lib.create_tweet_iterator(token, 
                                            token_secret,
                                            consumer_key,
                                            consumer_secret,
                                            bounding_box)

In [ ]:
tweets = [next(tweet_iterator) for _ in range(500)]

### Write list of tweets to JSON file on disk

In [ ]:
import json 

username = 'karl'

filename = lib.create_timestamped_filename(username)
with open(filename, 'w') as outfile:
    json.dump(tweets, outfile)
    
filename = lib.create_timestamped_filename(username)
with open(filename, 'w') as outfile:
    json.dump(tweets_2, outfile)    

### Write Tweet Files to S3, Step 1 - Create a Boto Client to S3

https://boto3.readthedocs.io

In [ ]:
s3_client = lib.create_boto_client()

In [ ]:
from os import listdir

In [ ]:
current_directory = listdir()
current_directory = [file for file in current_directory if ('tweets-karl' in file) and ('processed' not in file)]
current_directory

In [ ]:
from os import rename

### Write Tweet Files to S3, Step 2 - Write a file to S3

In [ ]:
S3_BUCKET = 'uclax-data-science'

for filename in current_directory:
    lib.write_file_to_S3(s3_client, filename, S3_BUCKET)
    old_filename = filename
    filename = filename.replace('.json', '')
    filename = filename + '-processed.json'
    rename(old_filename, filename)

In [ ]:
current_directory = listdir()
current_directory = [file for file in current_directory if 'tweets-karl' in file]
current_directory

### List Files on S3

In [ ]:
s3_files = lib.list_files_in_S3_bucket(s3_client, S3_BUCKET)
s3_files = [file for file in s3_files if 'tweets-karl' in file]
s3_files

## Write Tweets to Mongo

### Read an object from S3

In [ ]:
S3_BUCKET = 'uclax-data-science'
key = s3_files[1]

tweets_from_s3 = lib.read_object_from_S3(s3_client, key, S3_BUCKET)

### Write Tweets to Mongo

In [ ]:
from pymongo.errors import DuplicateKeyError

In [ ]:
collection_client = lib.create_mongo_client_to_database_collection('twitter', 'tweets')
for tweet in tweets_from_s3:
    try:
        collection_client.insert_one(tweet)
    except DuplicateKeyError as e:
        print(e)

In [ ]:
collection_client = lib.create_mongo_client_to_database_collection('twitter', 'tweets')
for tweet in tweets_from_s3:
    try:
        collection_client.insert_one(tweet)
    except DuplicateKeyError as e:
        print(e)

In [ ]:
for key in s3_files[2:]:
    for tweet in tweets_from_s3:
        try:
            collection_client.insert_one(tweet)
        except DuplicateKeyError as e:
            print(e)

## One More Time

In [ ]:
tweets = [next(tweet_iterator) for _ in range(50)]

filename = lib.create_timestamped_filename(username)
with open(filename, 'w') as outfile:
    json.dump(tweets, outfile)

In [ ]:
current_directory = listdir()
current_directory = [file for file in current_directory 
                     if ('tweets-joshua' in file) and ('processed' not in file)]
current_directory

In [ ]:
for filename in current_directory:
    lib.write_file_to_S3(s3_client, filename, S3_BUCKET)
    old_filename = filename
    filename = filename.replace('.json', '')
    filename = filename + '-processed.json'
    rename(old_filename, filename)

In [ ]:
s3_files = lib.list_files_in_S3_bucket(s3_client, S3_BUCKET)
s3_files = [file for file in s3_files if 'tweets-karl' in file]
s3_files

In [ ]:
s3_files[-1]

In [ ]:
key = s3_files[-1]

In [ ]:
tweets_from_s3 = lib.read_object_from_S3(s3_client, key, S3_BUCKET)

In [ ]:
collection_client.insert_many(tweets_from_s3)

# Automate It

In [ ]:
from datetime import datetime

In [ ]:
# TOKEN           = "12270642-8ggOVEjcQ7DAP2VCCZbXYoRtiOf0rngzrsyb82h7k"
# TOKEN_SECRET    = "85KnJJBbdvhZyGhOVLPwk47x2IAHaLEFGhNJdrHXo1Tv8"
# CONSUMER_KEY    = "Jt9H3ve228Tl8yafi4Ip8Iwbw"
# CONSUMER_SECRET = "xXxkq7QM1D7K1w0tFKRIYYviS2EQ288jzbbiwzEYYRQrMyvCDN"
# BOUNDING_BOX    = "-118.5137323688,34.0001996344,-118.4702449172,34.0331651696"

DATABASE        = 'twitter'
COLLECTION      = 'tweets'
USERNAME        = 'karl'
S3_BUCKET       = 'uclax-data-science'

database        = 'twitter'
collection      = 'tweets'
username        = 'karl'
s3_bucket       = 'uclax-data-science'

In [ ]:
tweet_iterator    = lib.create_tweet_iterator(token, 
                                            token_secret,
                                            consumer_key,
                                            consumer_secret,
                                            bounding_box)
s3_client         = lib.create_boto_client()
collection_client = lib.create_mongo_client_to_database_collection(DATABASE, COLLECTION)

In [ ]:
def timestamp():
    now = datetime.now().strftime('%D %H:%M:%S')
    print(now, end=' | ')
    print('Collecting 500 Tweets', end=' | ')
    
def collect_tweets(n=500):                            # SET NUMBER TO COLLECT IN EACH BATCH
    tweets = [next(tweet_iterator) for _ in range(n)]
    print('{} Tweets'.format(n), end=' | ')
    return tweets
    
def write_to_disk(tweets):
    filename = lib.create_timestamped_filename(USERNAME) 
    with open(filename, 'w') as outfile:
        json.dump(tweets, outfile)    
    print('Written to Disk', end=' | ')
    return filename
    
def write_to_S3(filename):   
    lib.write_file_to_S3(s3_client, filename, S3_BUCKET)
    old_filename = filename
    filename = filename.replace('.json', '')
    filename = filename + '-processed.json'
    rename(old_filename, filename)
    print('Written to S3', end=' | ')
    
def insert_to_mongo(key):
    tweets_from_s3 = lib.read_object_from_S3(s3_client, key, S3_BUCKET)    
    collection_client.insert_many(tweets_from_s3)
    print('Inserted to Mongo')

In [ ]:
while True:
    timestamp()
    tweets = collect_tweets()
    filename = write_to_disk(tweets)
    write_to_S3(filename)
    insert_to_mongo(filename)